# Implementing the vol-smoothing algorithm by Fengler[2009]

## Spline Smoothing Setup

Assume that we observe call option prices at strikes $a = u_0 < u_1 < u_2 < \ldots < u_n < u_{n+1} = b$. The points $u_i$ are called knot-points. A function $g$ defined on $[a,b]$ is called a cubic spline, if two conditions are satisfied:

(1) $g$ is a cubic polynomial.

(2) On each of the sub-intervals $(a,u_1), (u_1, u_2), \ldots, (u_n, b)$, $g\in C^2[a,b]$, that is, its first and second derivatives are continuous at each $u_i$.

Amongst many possible interpolants satisfying the above conditions, the smoothest possible curve that can be interpolates the given points, is the one that minimizes the roughness quantified by $\int_a^b g''^2(u)du$. Such a spline is called the **natural cubic spline**(NCS). The total energy is minimum. 

Define the banded matrix $Q \in \mathbb{R}^{n \times (n-2)}$ as:

\begin{equation*}
Q=\begin{bmatrix}
\frac{1}{h_{1}} &  &  &  &  & \\
-\frac{1}{h_{1}} -\frac{1}{h_{2}} & \frac{1}{h_{2}} &  &  &  & \\
\frac{1}{h_{2}} & -\frac{1}{h_{2}} -\frac{1}{h_{3}} & \ddots  &  &  & \\
 & \frac{1}{h_{3}} & \ddots  & \frac{1}{h_{j-1}} &  & \\
 &  & \ddots  & -\frac{1}{h_{j-1}} -\frac{1}{h_{j}} & \ddots  & \\
 &  &  & \frac{1}{h_{j}} & \ddots  & \frac{1}{h_{n-2}}\\
 &  &  &  & \ddots  & -\frac{1}{h_{n-2}} -\frac{1}{h_{n-1}}\\
 &  &  &  &  & \frac{1}{h_{n-1}}
\end{bmatrix}
\end{equation*}

and the symmetric banded matrix $R \in \mathbb{R}^{(n-2) \times (n-2)}$ as:


\begin{equation*}
R=\begin{bmatrix}
\frac{1}{3}( h_{1} +h_{2}) & \frac{1}{6} h_{2} &  &  &  &  & \\
\frac{1}{6} h_{2} & \frac{1}{3}( h_{2} +h_{3}) & \frac{1}{6} h_{3} &  &  &  & \\
 & \frac{1}{6} h_{3} & \frac{1}{3}( h_{3} +h_{4}) & \frac{1}{6} h_{4} &  &  & \\
 &  & \frac{1}{6} h_{4} & \ddots  &  &  & \\
 &  &  &  & \frac{1}{3}( h_{j-1} +h_{j}) & \frac{1}{6} h_{j} & \\
 &  &  &  & \frac{1}{6} h_{j} & \ddots  & \\
 &  &  &  &  &  & \frac{1}{3}( h_{n-2} +h_{n-1})
\end{bmatrix}
\end{equation*}

We define $g(u)$ in the functional form:

$$
g(u) = d_i(u - u_i)^3 + c_i(u - u_i)^2 + b_i(u - u_i) + a_i, \quad \forall 0 \leq i \leq n, u_i \leq u \leq u_{i+1} \tag{1}
$$

Thus, we have to derive the value of $4n + 4$ coefficients. 

$$
g_i = g(u_{i+1}), \quad \gamma_i = g''(u_i) \tag{2}
$$

Let $S_i(u)$ denote the piece in the $i$-th sub-interval. 

$$
\begin{align*}
S_i(u) &=  d_i(u - u_i)^3 + c_i(u - u_i)^2 + b_i(u - u_i) + a_i\\
S_i'(u) &= 3d_i(u - u_i)^2 + 2c_i(u - u_i) + b_i\\
S_i''(u) &= 6d_i(u - u_i) + 2c_i
\end{align*}
$$

So, $S_{i+1}(u_{i+1}) = g_{i+1} = a_{i+1}$, $S_{i+1}'(u_{i+1}) = b_{i+1}$ and $S_{i+1}''(u_{i+1}) = g''(u_{i+1}) = \gamma_{i+1} =  2c_{i+1}$. 

For an NCS, the first and second derivatives at the end-points are zero. Thus,

$c_0 = d_0 = c_{n} = d_{n} = 0$

Also, the quantity $(u_{i+1} - u_i)$ appears so often, that we let $h_i = u_{i+1} - u_i$ for $i=1,\ldots,n$.

Since we require the curve and its first and second derivatives to be continuous at the knot points, we require that:

$$
\begin{align*}
S_{i+1}(u_{i+1}) &= S_i(u_{i+1})\\
S_{i+1}'(u_{i+1}) &= S_i'(u_{i+1})\\
S_{i+1}''(u_{i+1}) &= S_i''(u_{i+1})\\
\end{align*}
$$

Consequently, we must have:

$$
a_{i+1} = d_i h_i^3 + c_i h_i^2 + b_i h_i + a_i \tag{3}
$$

$$
b_{i+1} = 3d_ih_i^2 + 2c_i h_i + b_i \tag{4}
$$

$$
2c_{i+1} = 6d_i h_i + 2c_i \tag{5}
$$

for each $i=0,\ldots,n$. Solving for $d_i$ in equation (5) and substituting this value in (3) and (4), we have:

$$
d_i = \frac{2c_{i+1} - 2c_i}{6h_i} = \frac{\gamma_{i+1} - \gamma_{i}}{6h_i}, \quad \forall i =1,\ldots,n \tag{7}
$$

$$
\begin{align*}
g_{i+1} &= \frac{2c_{i+1} - 2c_i}{6h_i} h_i^3 + c_i h_i^2 + b_i h_i + g_i\\
&= \frac{(2c_{i+1} - 2c_i)h_i^2}{6} + \frac{6c_i}{6}h_i^2 + b_i h_i + g_i \\
&= \frac{4c_i + 2c_{i+1}}{6} h_i^2 + b_i h_i + g_i \tag{8}
\end{align*}
$$

and

$$
\begin{align*}
b_{i+1} &= 3\left(\frac{2c_{i+1} - 2c_{i}}{6h_i}\right)h_i^2 + 2c_i h_i + b_i\\
&= (c_{i+1} - c_i)h_i + 2c_i h_i + b_i \\
&= (c_{i+1} + c_i)h_i + b_i \tag{9}
\end{align*}
$$

The final relationship is obtained by by solving equation (8) for $b_i$ :

$$
\begin{align*}
b_i = \frac{1}{h_i}(g_{i+1} - g_i) - \frac{4c_i + 2c_{i+1}}{6} h_i \tag{9}
\end{align*}
$$

and then with a reduction of index for $b_{i-1}$ yields:

$$
b_{i-1} = \frac{1}{h_{i-1}}(g_{i} - g_{i-1}) - \frac{4c_{i-1} + 2c_{i}}{6} h_{i-1} \tag{10}
$$

Substituting these values intom the equation derived from (9), with the index reduced from one, we get:

$$
\begin{align*}
b_i &= (c_i + c_{i-1})h_{i-1} + b_{i-1}\\
\frac{1}{h_i}(g_{i+1} - g_i) - \frac{4c_i + 2c_{i+1}}{6} h_i &= (c_i + c_{i-1})h_{i-1} + \frac{1}{h_{i-1}}(g_{i} - g_{i-1}) - \frac{4c_{i-1} + 2c_{i}}{6} h_{i-1}
\end{align*}
$$

We have:

$$
\begin{align*}
\frac{1}{h_i}(g_{i+1} - g_i) - \frac{1}{h_{i-1}}(g_i - g_{i-1}) &= \frac{4c_i + 2c_{i+1}}{6}h_i + (c_i + c_{i-1})h_{i-1} - \frac{4c_{i-1} + 2c_i}{6}h_{i-1}\\
&= \frac{4c_i + 2c_{i+1}}{6}h_i + \frac{(6c_i + 6c_{i-1})}{6}h_{i-1} - \frac{4c_{i-1} + 2c_i}{6}h_{i-1}\\
&= \frac{4c_i + 2c_{i+1}}{6}h_i + \frac{(4c_i + 2c_{i-1})}{6}h_{i-1}\\
&= \frac{2c_{i-1}}{6}h_{i-1} + \frac{4c_i}{6}(h_{i-1} + h_{i}) + \frac{2c_{i+1}}{6}h_i \tag{11}
\end{align*}
$$

Substituting $2c_i = \gamma_i$, we have:

$$
\frac{1}{h_i}g_{i+1} - \left(\frac{1}{h_i} + \frac{1}{h_{i-1}}\right)g_i + \frac{1}{h_{i-1}}g_{i-1} = \frac{\gamma_{i-1}}{6}h_{i-1} + \frac{2\gamma_i}{6}(h_{i-1} + h_{i}) + \frac{\gamma_{i+1}}{6}h_i \quad \forall i=2,\ldots,n-1  \tag{12}
$$

In matrix form:

$$
\begin{equation*}
\begin{bmatrix}
1/h_{1} & -( 1/h_{1} +1/h_{2}) & 1/h_{2} &  &  & \\
 & 1/h_{2} & -( 1/h_{2} +1/h_{3}) & 1/h_{3} &  & \\
 &  &  &  &  & \\
 \\
 &  &  & 1/h_{n-2} & -( 1/h_{n-2} +1/h_{n-1}) & 1/h_{n-1}
\end{bmatrix}\begin{bmatrix}
g_{1}\\
g_{2}\\
\vdots \\
g_{n}
\end{bmatrix} =\begin{bmatrix}
\frac{1}{3}( h_{1} +h_{2}) & \frac{1}{6} h_{2}  \\
 \frac{1}{6} h_{2} & \frac{1}{3}( h_{2} +h_{3}) & \frac{1}{6} h_{3}  \\
 & \frac{1}{6} h_{3} & \frac{1}{3}( h_{3} +h_{4}) & \frac{1}{6} h_{4} \\
 & & &  &\\
 \\
 & & & & \frac{1}{6} h_{n-2}& \frac{1}{3}( h_{n-2} +h_{n-1}) 
\end{bmatrix}
\begin{bmatrix}
\gamma_2 \\ 
\vdots \\
\gamma_{n-1}
\end{bmatrix}
\end{equation*}
$$

or 

$$
\mathbf{Q}^T \mathbf{g} = \mathbf{R} \mathbf{\gamma}
$$

where $\mathbf{Q}^T \in \mathbb{R}^{(n-2)\times n}$, $\mathbf{g} \in \mathbb{R}^n$, $\mathbf{R} \in \mathbb{R}^{(n-2) \times (n-2)}$, $\mathbf{\gamma} \in \mathbb{R}^{n-2} $. 

The roughness term captured by the integral $\int_{a}^{b} g''(u)^2 du$ is equal to the value of the quadratic form $\mathbf{\gamma}^T \mathbf{R} \mathbf{\gamma}$. 

Here is a proof sketch.

$$

\begin{equation*}
\begin{aligned}
\int _{a}^{b}\{g''( u)\}^{2} du & =\int _{a}^{b}\underbrace{g''( u)}_{u} \cdot \underbrace{g''( u) du}_{dv}\\
 & =g''( u) g'( u) |_{a}^{b} -\int _{a}^{b} g'( u) g'''( u) du\\
 & \left\{\text{ Integration by parts }\right\}\\
 & =g''( b) g'( b) \ -\ g''( a) g'( a) \ -\int _{a}^{b} g'( u) g'''( u) du\\
 & \{g''( a) =g''( b) =0\}\\
 & =-\int _{a}^{b} g'( u) g'''( u) du\\
 & =-\sum _{i=1}^{n-1} g'''\left( u_{i}^{+}\right)\int _{u_{i}}^{u_{i+1}} g'( u) du\\
 & \left\{\ g'''( u) \ \text{ is constant on all sub-intervals}\right\}\\
 & =\sum _{i=1}^{n-1}\frac{\gamma _{i+1} -\gamma _{i}}{h_{i}} \cdot ( g_{i} -g_{i+1})
\end{aligned}
\end{equation*}
$$

Since $\gamma_1 = \gamma_n = 0$, we have:

$$
\begin{equation*}
\begin{aligned}
\int _{a}^{b} g''( u)^{2} du & =\sum _{i=1}^{n-1}\frac{\gamma _{i+1} -\gamma _{i}}{h_{i}}\left( g_{i} -g_{i+1}\right)\\
 & =\frac{\gamma _{2}}{h_{1}}( g_{1} -g_{2}) +\frac{\gamma _{3} -\gamma _{2}}{h_{2}}( g_{2} -g_{3}) +\dotsc +\frac{\gamma _{n-1} -\gamma _{n-2}}{h_{n-2}}( g_{n-2} +g_{n-1}) +\frac{-\gamma _{n-1}}{h_{n-1}}( g_{n-1} -g_{n})\\
 & =\gamma _{2}\left(\frac{( g_{1} -g_{2})}{h_{1}} -\frac{( g_{2} -g_{3})}{h_{2}}\right) +\gamma _{3}\left(\frac{( g_{2} -g_{3})}{h_{3}} -\frac{( g_{3} -g_{4})}{h_{4}}\right) +\dotsc +\gamma _{n-1}\left(\frac{( g_{n-2} -g_{n-1})}{h_{n-2}} -\frac{( g_{n-1} -g_{n})}{h_{n-1}}\right)\\
 & =\sum _{j=2}^{n-1} \gamma _{j}\left(\frac{( g_{j-1} -g_{j})}{h_{j-1}} -\frac{( g_{j} -g_{j+1})}{h_{j}}\right)\\
 & =\mathbf{\gamma Q}^{T}\mathbf{g}\\
 & =\mathbf{\gamma R\gamma }
\end{aligned}
\end{equation*}
$$

Fengler's proposes an approach based on cubic spline smoothing of option prices rather than interpolation. Therefore, the input tdata do not have to be arbitrage free. We try solve for the smoothest possible spline, such that the weighted sum of squares:

$$
\sum_{i=1}^{n} w_i \left[y_i - g(u_i)\right]^2 + \lambda \mathbf{\gamma}^T \mathbf{R} \mathbf{\gamma}
$$

is minimized, subject to no-arbitrage constraints.

The exact quadratic programming problem formulation is stated in Fengler's paper.

## References
- [Arbitrage-free smoothing for volatility surfaces](https://www.researchgate.net/publication/46528351_Arbitrage-free_smoothing_of_the_implied_volatility_surface), *Matthias Fengler*
- [Non-parametric regression and generalized linear models](http://ndl.ethernet.edu.et/bitstream/123456789/39543/1/P.J.GREEN.pdf)

## Initialize the market-object builder

In [6]:
import datetime 
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import math

from py_volanalytics.models.mob.fengler_vol_surface_builder import FenglerVolSurfaceBuilder
from py_volanalytics.types.enums import TimeInfo, MarketDataServiceId, OptionType, OptionQuoteConvention, Currency
from py_volanalytics.market.time import Time
from py_volanalytics.market.option_quotes import OptionQuotes, OptionQuote
from py_volanalytics.market.forward_quotes import ForwardQuotes, ForwardQuote
from py_volanalytics.market.discounting_curve import DiscountingCurve
from py_volanalytics.valuation_framework.market_data import MarketDataService, MarketEnvironment

# Create an instance of the FenglerVolSurfaceBuilder
builder = FenglerVolSurfaceBuilder(symbol="SPX")

# Initialize the market object builder
builder.initialize()

# Query the builder for static dependencies
builder.get_static_dependencies(initialized_state=None)

# Query the builder for market dependencies
market_deps = builder.get_market_dependencies(initialized_state=None, reference_data=None)
market_deps

[TimeObjectId(_friendly_name=<MarketObjects.TIME: 'TIME'>, _time_info=<TimeInfo.TODAY: 2>),
 TimeObjectId(_friendly_name=<MarketObjects.TIME: 'TIME'>, _time_info=<TimeInfo.PV_DATE: 1>),
 OptionQuotesId(_friendly_name=<MarketObjects.OPTION_QUOTES: 'OPTION_QUOTES'>, _symbol='SPX'),
 ForwardQuotesId(_friendly_name=<MarketObjects.FORWARD_QUOTES: 'FORWARD_QUOTES'>, _symbol='SPX'),
 DiscountingCurveId(_friendly_name=<MarketObjects.DISCOUNTING_CURVE: 'DISCOUNTING_CURVE'>, _currency=<Currency.USD: 'USD'>, _collateral=<Currency.USD: 'USD'>)]

## Prepare the market data environment

In [7]:
# Construct time service
pv_date = Time.create(time_info=TimeInfo.PV_DATE, date=datetime.date(2022,3,4))
today = Time.create(time_info=TimeInfo.TODAY, date=datetime.date(2022,3,4))
time_service = MarketDataService.create(service_id=MarketDataServiceId.TIME_SERVICE, market_objects=[pv_date, today])

# Load the parquet file
option_data = pq.read_table("SPX_2022_03_04_10_01_00.parquet")
option_data = option_data.filter(pa.compute.greater(option_data['F'], 0.))
option_data = option_data.filter(pa.compute.less(option_data['T'], .5))

strikes = np.unique(np.array(option_data["K"]))
expiries = np.unique(np.array(option_data["T"]))
impl_vols_bid = np.zeros([len(expiries),len(strikes)])
impl_vols_ask = np.zeros([len(expiries),len(strikes)])
impl_vols = np.zeros([len(expiries),len(strikes)])
forwards = []
interest_rates = []
options = []
i = 0
for T in expiries:
    option_data_single_expiry = option_data.filter(pa.compute.equal(option_data["T"], T))
    for K in strikes:
        option_data_for_strike_maturity = option_data_single_expiry.filter(pa.compute.equal(option_data_single_expiry["K"], K))
        options.append(OptionQuote(option_type=OptionType.CALL_OPTION, strike_point=K, time_to_expiry=T, quote_convention=OptionQuoteConvention.IMPLIED_VOLATILITY, quote= option_data_for_strike_maturity["K"][0].as_py()))


    forwards.append(ForwardQuote(time_to_expiry=T, 
                           quote=option_data_for_strike_maturity["F"][0].as_py(),))

In [8]:
# Construct the OptionQuotes service
option_quotes = OptionQuotes.create(symbol="SPX", option_quotes=options)
option_quotes_service = MarketDataService.create(service_id=MarketDataServiceId.OPTION_QUOTES_SERVICE, market_objects=[option_quotes])

# Construct the ForwardQuotes service
forward_quotes_obj = ForwardQuotes.create(symbol="SPX", forward_quotes=forwards)
forward_quotes_service = MarketDataService.create(service_id=MarketDataServiceId.FORWARD_QUOTES_SERVICE, market_objects=[forward_quotes_obj])

# Construct the DiscountingCurve service
# I choose a dummy flat discount curve for the purposes of this exercise
dc = DiscountingCurve.flat(trade_ccy=Currency.USD, collateral_ccy=Currency.USD, rate=0.05)
discounting_curve_service = MarketDataService.create(service_id=MarketDataServiceId.DISCOUNTING_CURVE_SERVICE, market_objects=[dc])

# Construct the market enviroment
mktenv = MarketEnvironment.create([time_service, option_quotes_service, forward_quotes_service,discounting_curve_service])

TypeError: isinstance() argument 2 cannot be a parameterized generic

array([ 5, 12, 21, 32])